# Modelo Aberto de Jackson Aplicado à uma Estação de Tratamento de Água

## Objetivo

Criar um modelo que descreva o comportamento de uma estação de tratamento de água composta por 4 tratamentos: A, B, C e D. As etapas estão dispostas conforme o diagrama a seguir:

![Quadro](./imagens/Diagrama_Porto.PNG)

O fluxo de água é discretizado em volumes de 1 milhão de litros e a distribuição das chegadas seguem Poissons de parâmetro $\lambda_{j}$. O regime de serviço de cada unidade segue uma Exponencial de parâmetro $\mu_{i}$ com disciplina FIFO (First In First Out).

Cada unidade de tratamento possui 4 propriedades:
- Taxa de Chegada ($\lambda_{j}$): Volume de água em milhões de litros
- Taxa de Serviço ($\mu_{j}$): Volume de água que a unidade de tratamento tem capacidade de atender em milhões de litros
- Taxa de Ocupação ($\rho_{j}$): Utilização da unidade de tratamento

In [41]:

from enum import Enum
import pandas as pd

class TiposTratamento(Enum):
    A = 1
    B = 2
    C = 3
    D = 4

class Tratamento:
    def __init__(self, taxa_chegada: float, taxa_servico: float, tipo: TiposTratamento):
        self.taxa_chegada = taxa_chegada
        self.taxa_servico = taxa_servico
        self.taxa_ocupacao = taxa_chegada/taxa_servico
        self.tipo = tipo

def criar_tratamentos_df(tratamentos: list[Tratamento]) -> pd.DataFrame:
    df = pd.DataFrame({
            'tipo': [tratamento.tipo.name for tratamento in tratamentos],
            'taxas_chegada': [tratamento.taxa_chegada for tratamento in tratamentos],
            'taxas_servico': [tratamento.taxa_servico for tratamento in tratamentos],
            'taxas_ocupacao': [tratamento.taxa_ocupacao for tratamento in tratamentos]})
    df.index += 1
    return df

Para ilustrar os próximos conceitos, considere o seguinte cenário:

In [42]:
tratamentos = [
    Tratamento(10, 11, TiposTratamento.A),
    Tratamento(5, 7, TiposTratamento.B),
    Tratamento(6, 8, TiposTratamento.B),
    Tratamento(5, 7, TiposTratamento.C),
    Tratamento(8, 10, TiposTratamento.C),
    Tratamento(10, 11, TiposTratamento.D)
]

tratamentos_df = criar_tratamentos_df(tratamentos)
tratamentos_df

,tipo,taxas_chegada,taxas_servico,taxas_ocupacao
1,A,10,11,0.909091
2,B,5,7,0.714286
3,B,6,8,0.750000
4,C,5,7,0.714286
5,C,8,10,0.800000
6,D,10,11,0.909091


## Redes Abertas de Jackson

### Definição

O modelo de uma rede aberta de Jackson é formado por um conjunto de nós J que segue as seguintes regras:

1. As chegadas externas a cada nó ocorrem de acordo com um processo de Poisson de parâmetro $\lambda_{i}$, independente das outras chegadas e do serviço nos nós;
2. Cada nó segue a disciplina FIFO (First In First Out) segundo uma distribuição exponencial de parâmetro $\mu_{i}$;
3. Após o usuário completar seu serviço na estação, ele move-se para outro nó com probabilidade $P_{ij}$;
4. Todos os nós da rede poderão receber usuários externos de forma direta ou indireta;
5. O usuário servido em tem probabilidade positiva de sair da rede em uma ou mais etapas;

É adicionado ao sistema de equações um nó artificial J+1 que representa o exterior da rede. Neste trabalho o nó artificial recebeu o índice 0.


### Cálculo das Probabilidades $P_{ij}$

Podemos escrever as taxas de chegada $\lambda_{j}$ como:

$$
\lambda_{j} = \lambda_{0j} + \sum_{i=i}^{N} \mu_{i}P_{ij}, \quad j=1,2,3...N
$$

Exemplo: $\lambda_{1}$: chegada na Barra

$$
\lambda_{1} = \lambda_{01} + \sum_{i=i}^{N} \mu_{i}P_{i1}, \quad i=1,2,3,4,5,6
$$

$\lambda_{01} = 10$, porém pelo diagrama pode-se notar que $P_{i1} = 0$ para qualquer $i$, pois a Barra não recebe navios vindo das próximas etapas. Portanto,

$$
\lambda_{1} = \lambda_{01} = 10
$$

Exemplo: $\lambda_{5}$: chegada no Cais 2

$$
\lambda_{5} = \lambda_{05} + \sum_{i=i}^{N} \mu_{i}P_{i5}, \quad i=1,2,3,4,5,6
$$

Como apenas a Barra recebe navios de fora do sistema, $\lambda_{05} = 0$. O Cais 2 recebe apenas navios vindos dos nós de Manobras, ou seja, $i = \{2,3\}$. Para qualquer outro valor de $i$, $P_{i5} = 0$

$$
\lambda_{5} = \mu_{2}p_{25} + \mu_{3}P_{35}
$$

Portanto, as taxas de chegadas em função das probabilidades e taxas de serviço dos demais nós são dadas por:

$$
\lambda_{2} = \mu_{1} P_{12} \\
\lambda_{3} = \mu_{1} P_{13} \\
\lambda_{4} = \mu_{2} P_{24} + \mu_{3} P_{34} \\
\lambda_{5} = \mu_{2} P_{25} + \mu_{3} P_{35} \\
\lambda_{6} = \mu_{4} P_{46} + \mu_{5} P_{56} \\
$$

Como o objetivo é encontrar as probabilidades de um navio partir de um nó $i$ para outro nó $j$, precisamos de mais equações para formar o sistema possível e determinado. Para isso, usamos as taxas de serviço dos nós que são o resultado da soma das probabilidades dos caminhos que os navios podem tomar, ou seja:

$$
\mu_{1} = \mu_{1} P_{12} + \mu_{1} P_{13} \\
\mu_{2} = \mu_{2} P_{24} + \mu_{2} P_{25} \\
\mu_{3} = \mu_{3} P_{34} + \mu_{3} P_{35} \\
\mu_{4} = \mu_{4} P_{46} \\
\mu_{5} = \mu_{5} P_{56} \\
\mu_{6} = \mu_{6} P_{60}
$$

Logo, podemos escrever o sistema na forma matricial:

$$
\begin{gather}
    \begin{bmatrix}
        \lambda_{01} & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
        0 & \mu_{1} & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
        0 & 0 & \mu_{1} & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
        0 & 0 & 0 & \mu_{2} & 0 & \mu_{3} & 0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 & \mu_{2} & 0 & \mu_{3} & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 & 0 & 0 & 0 & \mu_{4} & \mu_{5} & 0 \\
        0 & \mu_{1} & \mu_{1} & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
        0 & 0 & 0 & \mu_{2} & \mu_{2} & 0 & 0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 & 0 & \mu_{3} & \mu_{3} & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 & 0 & 0 & 0 & \mu_{4} & 0 & 0 \\
        0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \mu_{5} & 0 \\
        0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \mu_{6}
    \end{bmatrix}
    \begin{bmatrix}
        P_{01} \\P_{12} \\ P_{13} \\ P_{24} \\ P_{25} \\ P_{34} \\ P_{35} \\ P_{46} \\ P_{56} \\ P_{60} \\
    \end{bmatrix}
    =
    \begin{bmatrix}
        \lambda_{1} \\ \lambda_{2} \\ \lambda_{3} \\ \lambda_{4} \\ \lambda_{5} \\ \lambda_{6} \\ \mu_{1} \\ \mu_{2} \\ \mu_{3} \\ \mu_{4} \\ \mu_{5} \\ \mu_{6} 
    \end{bmatrix}
\end{gather}
$$

Isolando o vetor de probabilidades, temos:


$$
\begin{gather}
    \begin{bmatrix}
        P_{01} \\ P_{12} \\ P_{13} \\ P_{24} \\ P_{25} \\ P_{34} \\ P_{35} \\ P_{46} \\ P_{56} \\ P_{60} \\ 
    \end{bmatrix}
    =
    \begin{bmatrix}
        \lambda_{01} & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
        0 & \mu_{1} & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
        0 & 0 & \mu_{1} & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
        0 & 0 & 0 & \mu_{2} & 0 & \mu_{3} & 0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 & \mu_{2} & 0 & \mu_{3} & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 & 0 & 0 & 0 & \mu_{4} & \mu_{5} & 0 \\
        0 & \mu_{1} & \mu_{1} & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
        0 & 0 & 0 & \mu_{2} & \mu_{2} & 0 & 0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 & 0 & \mu_{3} & \mu_{3} & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 & 0 & 0 & 0 & \mu_{4} & 0 & 0 \\
        0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \mu_{5} & 0 \\
        0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \mu_{6}
    \end{bmatrix}^{-1}
    \begin{bmatrix}
        \lambda_{1} \\ \lambda_{2} \\ \lambda_{3} \\ \lambda_{4} \\ \lambda_{5} \\ \lambda_{6} \\ \mu_{1} \\ \mu_{2} \\ \mu_{3} \\ \mu_{4} \\ \mu_{5} \\ \mu_{6} 
    \end{bmatrix}
\end{gather}
$$


In [43]:

import numpy as np
import utils
from IPython.display import Latex

vetor_probabilidades_latex = r'\begin{bmatrix} P_{01} \\P_{12} \\ P_{13} \\ P_{24} \\ P_{25} \\ P_{34} \\ P_{35} \\ P_{46} \\ P_{56} \\ P_{60} \\ \end{bmatrix}'

def criar_vetor_taxas(taxas_chegada: list[float], taxas_servico: list[float]) -> np.ndarray:
    return np.vstack(np.append(taxas_chegada, taxas_servico))

def criar_matriz_de_roteamento(taxas_chegada: list[float], taxas_servico: list[float]) -> np.ndarray:
    return np.array([
                [taxas_chegada[1], *np.zeros(9)],
                [0, taxas_servico[1], *np.zeros(8)],
                [*np.zeros(2), taxas_servico[1], *np.zeros(7)],
                [*np.zeros(3), taxas_servico[2], 0, taxas_servico[3], *np.zeros(4)],
                [*np.zeros(4), taxas_servico[2], 0, taxas_servico[3], *np.zeros(3)],
                [*np.zeros(7), taxas_servico[4], taxas_servico[5], 0],
                [0, taxas_servico[1], taxas_servico[1], *np.zeros(7)],
                [*np.zeros(3), taxas_servico[2], taxas_servico[2], *np.zeros(5)],
                [*np.zeros(5), taxas_servico[3], taxas_servico[3], *np.zeros(3)],
                [*np.zeros(7), taxas_servico[4], *np.zeros(2)],
                [*np.zeros(8), taxas_servico[5], 0],
                [*np.zeros(9), taxas_servico[6]],
    ])

def criar_equacao_trafego_isolando_P_latex(matriz_taxas_servico: np.ndarray, vetor_taxas: np.ndarray) -> Latex:
    return Latex(r'\begin{gather}' +
                vetor_probabilidades_latex +
                r'=' +
                utils.array_to_latex(matriz_taxas_servico,'') + r'^{-1}' +
                utils.array_to_latex(vetor_taxas, '') +
                r'\end{gather}')

def criar_equacao_P_latex(vetor_probabilidades: np.ndarray) -> Latex:
    return Latex(r'\begin{gather}' +
                vetor_probabilidades_latex +
                r'=' +
                utils.array_to_latex(vetor_probabilidades, '') +
                r'\end{gather}')


taxas_chegada = tratamentos_df.taxas_chegada
taxas_servico = tratamentos_df.taxas_servico
vetor_taxas = criar_vetor_taxas(taxas_chegada, taxas_servico)
matriz_taxas_servico = criar_matriz_de_roteamento(taxas_chegada, taxas_servico)

criar_equacao_trafego_isolando_P_latex(matriz_taxas_servico, vetor_taxas)


<IPython.core.display.Latex object>

In [44]:
vetor_probabilidades = np.linalg.pinv(matriz_taxas_servico).dot(vetor_taxas)

criar_equacao_P_latex(vetor_probabilidades)

<IPython.core.display.Latex object>

Portanto, as Probabilidades $P_{ij}$ para os nós no conjunto J são:

In [45]:
def criar_probabilidades_df(vetor_probabilidades: np.ndarray) -> pd.DataFrame:
      df = pd.DataFrame({
            'origens': [0, 1, 1, 2, 2, 3, 3, 4, 5, 6],
            'destinos': [1, 2, 3, 4, 5, 4, 5, 6, 6, 0],
            'probabilidades': vetor_probabilidades.T[0]
      })
      return df.groupby(['origens', 'destinos']).sum()

probabilidades_df = criar_probabilidades_df(vetor_probabilidades)

probabilidades_df

probabilidades
origens destinos                
0       1               1.000000
1       2               0.454545
        3               0.545455
2       4               0.371365
        5               0.557206
3       4               0.362555
        5               0.574945
4       6               0.666667
5       6               0.766667
6       0               1.000000

## Medidas de Desempenho

As medidas de desempenho de interesse são o volume de água no sistema e o tempo necessário para aplicar todos os tratamentos. Cada unidade de tratamento $i$ funciona como um serviço M/M/1, assim, o volume de água $L_{i}$ na estação e o tempo $W_{i}$ podem ser calculados por:

$$
L_{i} = \frac{\lambda_{i}}{\mu_{i} - \lambda_{i}} \\
W_{i} = \frac{1}{\mu_{i} - \lambda_{i}}
$$

In [46]:
tratamentos_df['volume_esperado'] = tratamentos_df.taxas_chegada / (tratamentos_df.taxas_servico - tratamentos_df.taxas_chegada)
tratamentos_df['tempo_esperado'] = 1 / (tratamentos_df.taxas_servico - tratamentos_df.taxas_chegada)

tratamentos_df

,tipo,taxas_chegada,taxas_servico,taxas_ocupacao,volume_esperado,tempo_esperado
1,A,10,11,0.909091,10.0,1.0
2,B,5,7,0.714286,2.5,0.5
3,B,6,8,0.750000,3.0,0.5
4,C,5,7,0.714286,2.5,0.5
5,C,8,10,0.800000,4.0,0.5
6,D,10,11,0.909091,10.0,1.0


Uma vez que a rede aberta de Jackson se comporta como uma coleção de serviços M/M/1 independentes, o volume total L e o tempo total no sistema W pode ser calculados por:

$$
L = \sum_{i=1}^{N} L_{i} \\
W = \sum_{i=1}^{N} L_{i}
$$

In [47]:
L = tratamentos_df.volume_esperado.sum()

Latex(f'L = {L} \\quad \\text{{milhões de litros}}')

<IPython.core.display.Latex object>

In [48]:
W = tratamentos_df.tempo_esperado.sum()

Latex(f'W = {W} \\quad \\text{{horas}}')

<IPython.core.display.Latex object>

É possível avaliar como mudanças observadas no sistema impactam nas medidas de desempenho. Por exemplo, diminuindo as taxas de serviço para $\mu_{j} = \lambda_{j} + 1$:

In [49]:
tratamentos = [
    Tratamento(10, 11, TiposTratamento.A),
    Tratamento(5, 6, TiposTratamento.B),
    Tratamento(6, 7, TiposTratamento.B),
    Tratamento(5, 6, TiposTratamento.C),
    Tratamento(8, 9, TiposTratamento.C),
    Tratamento(10, 11, TiposTratamento.D)
]

tratamentos_df = criar_tratamentos_df(tratamentos)
tratamentos_df['volume_esperado'] = tratamentos_df.taxas_chegada / (tratamentos_df.taxas_servico - tratamentos_df.taxas_chegada)
tratamentos_df['tempo_esperado'] = 1 / (tratamentos_df.taxas_servico - tratamentos_df.taxas_chegada)

tratamentos_df

,tipo,taxas_chegada,taxas_servico,taxas_ocupacao,volume_esperado,tempo_esperado
1,A,10,11,0.909091,10.0,1.0
2,B,5,6,0.833333,5.0,1.0
3,B,6,7,0.857143,6.0,1.0
4,C,5,6,0.833333,5.0,1.0
5,C,8,9,0.888889,8.0,1.0
6,D,10,11,0.909091,10.0,1.0


In [50]:
L = tratamentos_df.volume_esperado.sum()
W = tratamentos_df.tempo_esperado.sum()


Latex(r'\begin{gather}' +
    f'L = {L} \\quad \\text{{milhões de litros}}' +
    r'\\' +
    f'W = {W} \\quad \\text{{horas}}' +
    r'\end{gather}')

<IPython.core.display.Latex object>